# Lesson 4: Auto-merging Retrieval

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: b94e5741-5552-409c-a651-bdb1af379d2b
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Auto-merging retrieval setup

In [5]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [7]:
nodes = node_parser.get_nodes_from_documents([document])

In [8]:
from llama_index.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

But this became less important as numerical linear algebra libraries matured.
Deep learning is still an emerging technology, so when you train a neural network and the 
optimization algorithm struggles to converge, understanding the math behind gradient 
descent, momentum, and the Adam  optimization algorithm will help you make better decisions. 
Similarly, if your neural network does something funny — say, it makes bad predictions on 
images of a certain resolution, but not others — understanding the math behind neural network 
architectures puts you in a better position to figure out what to do.
Of course, I also encourage learning driven by curiosity.


In [9]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

PAGE 12Should You 
Learn Math to 
Get a Job in AI? CHAPTER 3
LEARNING

PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? It’s always nice to know more math! But there’s so much to 
learn that, realistically, it’s necessary to prioritize. Here’s how you might go about strengthening 
your math background.
To figure out what’s important to know, I find it useful to ask what you need to know to make 
the decisions required for the work you want to do. At DeepLearning.AI, we frequently ask, 
“What does someone need to know to accomplish their goals?” The goal might be building a 
machine learning model, architecting a system, or passing a job interview.
Understanding the math behind algorithms you use is often helpful, since it enables you to 
debug them. But the depth of knowledge that’s useful changes over time. As machine learning 
techniques mature and become more reliable and turnkey, they require less debugging, and a 
shallower understand

### Building the index

In [10]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [11]:
from llama_index import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

In [12]:
from llama_index import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [13]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )


### Defining the retriever and running the query engine

In [14]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [15]:
auto_merging_response = auto_merging_engine.query(
    "What is the importance of networking in AI?"
)

In [16]:
from llama_index.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Networking is important in AI because it allows individuals to build a strong professional network and community. This network can provide valuable information, help with career advancement, and offer support and advice when needed. By connecting with others in the AI community, individuals can also increase their visibility and recognition for their expertise. Additionally, networking can lead to referrals for potential job opportunities. Building a community and networking within it can be more beneficial than simply focusing on personal networking efforts.

## Putting it all Together

In [17]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [18]:
from llama_index.llms import OpenAI

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index",
)


In [19]:
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

## TruLens Evaluation

In [20]:
from trulens_eval import Tru

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Two layers

In [21]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)

In [22]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [23]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [24]:
eval_questions = []
with open('generated_questions.text', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [25]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [26]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

> Merging 1 nodes into parent node.
> Parent node id: ac87675c-c65d-477b-9683-165501d5814a.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: c3a3a612-ff05-4f34-9e91-ed8a1f496bf7.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: 6e7e014e-57f4-42ed-9f38-1e33eb762d55.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: 20aab00d-7a6b-4ec9-a5b0-23b123312b08.
> Parent node text: PAGE 23Each project is only one step on a longer journey, hopefully one that has a positive impac...

> Merging 1 nodes into parent node.
> Parent node id: 0d803cbb-cf26-4c4b-843e-0777996b6efe.
> Parent node text: PAGE 29If you’re preparing to sw

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


> Merging 1 nodes into parent node.
> Parent node id: 5449677d-460c-44b2-b8ed-5f01d4ea4f78.
> Parent node text: PAGE 27There’s a lot we don’t know about the future: When will we cure Alzheimer’s disease? Who w...

> Merging 2 nodes into parent node.
> Parent node id: ec7988bb-e816-4550-b71c-31b8f2dffe32.
> Parent node text: PAGE 20Working on projects requires making tough choices about what to build and how to go 
about...

> Merging 1 nodes into parent node.
> Parent node id: 78c56ac7-d9af-4a98-80cd-494eadf8abc7.
> Parent node text: PAGE 18It goes without saying that we should only work on projects that are responsible, ethical,...

> Merging 1 nodes into parent node.
> Parent node id: 6e7e014e-57f4-42ed-9f38-1e33eb762d55.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: dadc9756-1742-437c-a8ef-3636c16f9f96.
> Parent node text: PAGE 36Keys to Building a Career

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


> Merging 2 nodes into parent node.
> Parent node id: ec7988bb-e816-4550-b71c-31b8f2dffe32.
> Parent node text: PAGE 20Working on projects requires making tough choices about what to build and how to go 
about...

> Merging 1 nodes into parent node.
> Parent node id: 78c56ac7-d9af-4a98-80cd-494eadf8abc7.
> Parent node text: PAGE 18It goes without saying that we should only work on projects that are responsible, ethical,...

> Merging 1 nodes into parent node.
> Parent node id: 1906c96d-0c11-4f52-99f9-2876a84ae850.
> Parent node text: PAGE 22Over the course of a career, you’re likely to work on projects in succession, each growing...

> Merging 1 nodes into parent node.
> Parent node id: c53c3a62-dbe9-4d28-9dcb-9fc7e63a146c.
> Parent node text: PAGE 7These phases apply in a wide 
range of professions, but AI 
involves unique elements.
For e...

> Merging 1 nodes into parent node.
> Parent node id: 5eab4b7b-6473-46a4-ba8f-9bfa0aff424a.
> Parent node text: PAGE 15One of the most important

...

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICal

> Merging 1 nodes into parent node.
> Parent node id: 97c5abe8-e8e4-4700-b503-13132a9fcd7e.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 5eab4b7b-6473-46a4-ba8f-9bfa0aff424a.
> Parent node text: PAGE 15One of the most important skills of an AI architect is the ability to identify ideas that ...

> Merging 1 nodes into parent node.
> Parent node id: 09106f90-2fd7-461e-b8d3-09a1190e3390.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...

> Merging 1 nodes into parent node.
> Parent node id: bec25e0c-de0d-4808-9ca4-f2c6220877fa.
> Parent node text: PAGE 16Determine milestones. Once you’ve deemed a project sufficiently 
valuable, the next step i...

> Merging 1 nodes into parent node.
> Parent node id: 20aab00d-7a6b-4ec9-a5b0-23b123312b08.
> Parent node text: PAGE 23Each project is only one 

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


> Merging 1 nodes into parent node.
> Parent node id: e50feeb9-eb83-4515-8b6b-215be8d79188.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: d1f2b35f-131c-4916-972b-21cffbeaa2e5.
> Parent node text: PAGE 37Overcoming Imposter 
SyndromeCHAPTER 11

> Merging 1 nodes into parent node.
> Parent node id: 97c5abe8-e8e4-4700-b503-13132a9fcd7e.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 9aa5726d-1c5d-43a5-b856-d0a657d22b98.
> Parent node text: PAGE 39My three-year-old daughter (who can barely count to 12) regularly tries to teach things to...

> Merging 1 nodes into parent node.
> Parent node id: ee0fb7f7-9cfb-45f5-b21e-c96bc368f05a.
> Parent node text: PAGE 4Coding AI Is the New Literacy
Today we take it for granted that many people know

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Validation error: 1 validation error for Rating
rating
  Value error, Rating must be between 0 and 10 [type=value_error, input_value=428571428571429, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


In [27]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
app_0,0.918182,0.864688,0.368254,14.75,0.0


In [28]:
Tru().run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


Dashboard started at http://10.10.1.8:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Three layers

In [29]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048,512,128],
)

In [30]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)


Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


In [31]:
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [32]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

> Merging 4 nodes into parent node.
> Parent node id: 8936fb89-49f2-4ee6-b8ca-afa47e5c41e7.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 5 nodes into parent node.
> Parent node id: 23bca79a-50d3-45b4-9b3b-6602af47a16e.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: 79ed3fc7-3136-4633-8b3d-1e58f5d1df6d.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: 3da4f7b5-7cd5-4209-9a01-7ee59ea64d29.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...



Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICal

> Merging 5 nodes into parent node.
> Parent node id: 0426c530-46e9-45f6-bea5-cde475c3d873.
> Parent node text: PAGE 27There’s a lot we don’t know about the future: When will we cure Alzheimer’s disease? Who w...

> Merging 1 nodes into parent node.
> Parent node id: 5b30675c-9ea5-472b-977c-647445e5f3fd.
> Parent node text: PAGE 27There’s a lot we don’t know about the future: When will we cure Alzheimer’s disease? Who w...



Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICal

> Merging 4 nodes into parent node.
> Parent node id: 67ce58ca-9e65-4a6b-b7ff-fc3d8a8c06d5.
> Parent node text: PAGE 20Working on projects requires making tough choices about what to build and how to go 
about...

> Merging 2 nodes into parent node.
> Parent node id: d7729928-e097-4ec4-9b1d-171572fde6d2.
> Parent node text: But when committing to a direction means making a costly investment or entering a one-
way door  ...

> Merging 2 nodes into parent node.
> Parent node id: 518f0bb7-7527-4558-a934-206add68dab0.
> Parent node text: PAGE 20Working on projects requires making tough choices about what to build and how to go 
about...



...

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICal

> Merging 2 nodes into parent node.
> Parent node id: d07240be-6036-4834-97eb-308f32b8773a.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: d021b773-e983-423f-95fa-32c0757265f6.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


> Merging 4 nodes into parent node.
> Parent node id: b75ecd31-20d4-4ae7-af23-f9bc3192e300.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 3 nodes into parent node.
> Parent node id: 21540d98-5413-4da6-bbd6-7b9a01d37d4f.
> Parent node text: PAGE 39My three-year-old daughter (who can barely count to 12) regularly tries to teach things to...

> Merging 1 nodes into parent node.
> Parent node id: 3648118a-a087-4d35-a013-cd6306613ceb.
> Parent node text: PAGE 37Overcoming Imposter 
SyndromeCHAPTER 11

> Merging 1 nodes into parent node.
> Parent node id: e0366e4e-5746-4590-b70d-c352425d450d.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 54b13c51-e070-4d56-b8f7-ef62d8d23587.
> Parent node text: PAGE 39My three-year-old daughter (who can barely count to 12) regularly tries to teac

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


In [33]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
app_0,0.925000,0.886753,0.367361,14.75,0.0
app_1,0.865217,0.883682,0.365238,14.75,0.0


In [34]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.10.1.8:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICal